In [ ]:
# 🏗️ Install sqlite3 CLI in the Colab environment
!apt-get update
!apt-get install -y sqlite3


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,762 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,964 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages 

In [ ]:
db_path = "/content/drive/MyDrive/big_project/inventory.db"       # corrupted source
new_db_path = "/content/drive/MyDrive/big_project/inventory_fixed.db"  # recovered target


In [ ]:
import sqlite3
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
integrity = cursor.execute("PRAGMA integrity_check;").fetchall()
conn.close()
print("Integrity Check:", integrity)


Integrity Check: [('*** in database main ***\nMain freelist: freelist leaf count too big on page 6891\nMain freelist: invalid page number 218103808\nOn tree page 4656 cell 0: 2nd reference to page 4657\nPage 4942 is never used\nPage 5202 is never used\nPage 5562 is never used\nPage 5872 is never used\nPage 5874 is never used\nPage 5875 is never used\nPage 5876 is never used\nPage 5877 is never used\nPage 5878 is never used\nPage 5879 is never used\nPage 5880 is never used\nPage 5881 is never used\nPage 5882 is never used\nPage 5883 is never used\nPage 5884 is never used\nPage 5885 is never used\nPage 5886 is never used\nPage 5887 is never used\nPage 5888 is never used\nPage 5889 is never used\nPage 5890 is never used\nPage 5891 is never used\nPage 5892 is never used\nPage 5893 is never used\nPage 5894 is never used\nPage 5895 is never used\nPage 5896 is never used\nPage 5897 is never used\nPage 5898 is never used\nPage 5899 is never used\nPage 5900 is never used\nPage 5901 is never use

In [ ]:
# ✅ Make sure sqlite3 CLI is installed (run once)
!apt-get update -qq
!apt-get install -y sqlite3

# 🔧 Paths
db_path = "/content/drive/MyDrive/big_project/inventory.db"
new_db_path = "/content/drive/MyDrive/big_project/inventory_fixed.db"

# 💾 Dump the corrupted DB to a SQL file
!sqlite3 "{db_path}" ".dump" > dump.sql

# 📦 Rebuild a clean DB from the dump
!sqlite3 "{new_db_path}" < dump.sql

print(f"🚀✅ Rebuild complete! New database at: {new_db_path}")


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
sqlite3 is already the newest version (3.37.2-2ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
🚀✅ Rebuild complete! New database at: /content/drive/MyDrive/big_project/inventory_fixed.db


In [ ]:
import pandas as pd
import os
from sqlalchemy import create_engine
import logging
import time


import importlib
importlib.reload(logging)

log_dir = '/content/drive/MyDrive/big_project/logs/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)


logging.basicConfig(
    filename='/content/drive/MyDrive/big_project/logs/ingestion_db.log',
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode='a'
)

engine = create_engine('sqlite:///drive/MyDrive/big_project/inventory.db')



def ingest_db(df, table_name, engine):
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)

def load_raw_data():
    start = time.time()
    for file in os.listdir('drive/MyDrive/big_project/data/'):
        if '.csv' in file:
            df = pd.read_csv(f'drive/MyDrive/big_project/data/{file}')
            logging.info(f'Ingesting {file} in db')
            ingest_db(df, file[:-4], engine)
            print(df.shape)

    end = time.time()
    total_time = (end - start) / 60
    logging.info('-------Ingestion Complete------')
    print(total_time)
    logging.info(f'Total Time taken: {total_time} minutes')

# if _name_ == '_main_':
load_raw_data()

(206529, 9)
(2372474, 16)
(5543, 10)
(224489, 9)
(152945, 14)
(12261, 9)
1.7429384469985962


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import sqlite3

In [3]:

conn = sqlite3.connect('/content/drive/MyDrive/big_project/inventory.db')


In [4]:
tables = pd.read_sql_query("SELECT name FROM sqlite_master where type = 'table' ",conn)
tables

,name
0,begin_inventory
1,purchases
2,vendor_invoice
3,end_inventory
4,sales
5,purchase_prices


In [ ]:
for table in tables['name']:
  if table != 'purchase_prices':
    print('-'*50,f'{table}','-'*50)
    print(f"{table} count = ", pd.read_sql(f"select count(*) as count from {table}",conn)['count'].values[0])
    display(pd.read_sql(f"select * from {table} limit 5",conn))

-------------------------------------------------- vendor_invoice --------------------------------------------------
vendor_invoice count =  5543


,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,105,ALTAMAR BRANDS LLC,2024-01-04,8124,2023-12-21,2024-02-16,6,214.26,3.47,None
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57,None
2,388,ATLANTIC IMPORTING COMPANY,2024-01-09,8169,2023-12-24,2024-02-16,5,106.60,4.61,None
3,480,BACARDI USA INC,2024-01-12,8106,2023-12-20,2024-02-05,10100,137483.78,2935.20,None
4,516,BANFI PRODUCTS CORP,2024-01-07,8170,2023-12-24,2024-02-12,1935,15527.25,429.20,None


-------------------------------------------------- end_inventory --------------------------------------------------
end_inventory count =  224489


,InventoryId,Store,City,Brand,Description,Size,onHand,Price,endDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,11,12.99,2024-12-31
1,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,7,36.99,2024-12-31
2,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,7,38.99,2024-12-31
3,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750mL,4,34.99,2024-12-31
4,1_HARDERSFIELD_75,1,HARDERSFIELD,75,Three Olives Tomato Vodka,750mL,7,14.99,2024-12-31


-------------------------------------------------- sales --------------------------------------------------
sales count =  152945


,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-01,750.0,1,0.79,12546.0,JIM BEAM BRANDS COMPANY
1,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,2024-01-02,750.0,1,1.57,12546.0,JIM BEAM BRANDS COMPANY
2,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-03,750.0,1,0.79,12546.0,JIM BEAM BRANDS COMPANY
3,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,14.49,14.49,2024-01-08,750.0,1,0.79,12546.0,JIM BEAM BRANDS COMPANY
4,1_HARDERSFIELD_1005,1,1005,Maker's Mark Combo Pack,375mL 2 Pk,2,69.98,34.99,2024-01-09,375.0,1,0.79,12546.0,JIM BEAM BRANDS COMPANY


-------------------------------------------------- begin_inventory --------------------------------------------------
begin_inventory count =  206529


,InventoryId,Store,City,Brand,Description,Size,onHand,Price,startDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,2024-01-01
1,1_HARDERSFIELD_60,1,HARDERSFIELD,60,Canadian Club 1858 VAP,750mL,7,10.99,2024-01-01
2,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,6,36.99,2024-01-01
3,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,3,38.99,2024-01-01
4,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750mL,6,34.99,2024-01-01


-------------------------------------------------- purchases --------------------------------------------------
purchases count =  0


,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification


In [8]:
for table in tables['name']:
  print(f"{table}:")
  print(f"    count of records", pd.read_sql_query(f"select count (*) as count from {table} ",conn)['count'].values[0])
  display(pd.read_sql_query(f"select * from {table} limit 5",conn))

begin_inventory:
    count of records 206529


,InventoryId,Store,City,Brand,Description,Size,onHand,Price,startDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,2024-01-01
1,1_HARDERSFIELD_60,1,HARDERSFIELD,60,Canadian Club 1858 VAP,750mL,7,10.99,2024-01-01
2,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,6,36.99,2024-01-01
3,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,3,38.99,2024-01-01
4,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750mL,6,34.99,2024-01-01


purchases:
    count of records 2372474


,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,69_MOUNTMEND_8412,69,8412,Tequila Ocho Plata Fresno,750mL,105,ALTAMAR BRANDS LLC,8124,2023-12-21,2024-01-02,2024-01-04,2024-02-16,35.71,6,214.26,1
1,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,4,37.40,1
2,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,9.41,5,47.05,1
3,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,6,56.10,1
4,76_DONCASTER_2034,76,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,8169,2023-12-24,2024-01-02,2024-01-09,2024-02-16,21.32,5,106.60,1


vendor_invoice:
    count of records 5543


...

,InventoryId,Store,City,Brand,Description,Size,onHand,Price,endDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,11,12.99,2024-12-31
1,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,7,36.99,2024-12-31
2,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,7,38.99,2024-12-31
3,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750mL,4,34.99,2024-12-31
4,1_HARDERSFIELD_75,1,HARDERSFIELD,75,Three Olives Tomato Vodka,750mL,7,14.99,2024-12-31


sales:
    count of records 152945


,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-01,750.0,1,0.79,12546.0,JIM BEAM BRANDS COMPANY
1,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,2024-01-02,750.0,1,1.57,12546.0,JIM BEAM BRANDS COMPANY
2,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-03,750.0,1,0.79,12546.0,JIM BEAM BRANDS COMPANY
3,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,14.49,14.49,2024-01-08,750.0,1,0.79,12546.0,JIM BEAM BRANDS COMPANY
4,1_HARDERSFIELD_1005,1,1005,Maker's Mark Combo Pack,375mL 2 Pk,2,69.98,34.99,2024-01-09,375.0,1,0.79,12546.0,JIM BEAM BRANDS COMPANY


purchase_prices:
    count of records 12261


,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,58,Gekkeikan Black & Gold Sake,12.99,750mL,750,1,9.28,8320,SHAW ROSS INT L IMP LTD
1,62,Herradura Silver Tequila,36.99,750mL,750,1,28.67,1128,BROWN-FORMAN CORP
2,63,Herradura Reposado Tequila,38.99,750mL,750,1,30.46,1128,BROWN-FORMAN CORP
3,72,No. 3 London Dry Gin,34.99,750mL,750,1,26.11,9165,ULTRA BEVERAGE COMPANY LLP
4,75,Three Olives Tomato Vodka,14.99,750mL,750,1,10.94,7245,PROXIMO SPIRITS INC.


In [ ]:
purchases = pd.read_sql_query(f"select * from purchases where VendorNumber = 4466",conn)
purchases[:5]

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification


In [ ]:
purchase_prices = pd.read_sql_query(f"select * from purchase_prices where VendorNumber = 4466",conn)
purchase_prices

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,5215,TGI Fridays Long Island Iced,12.99,1750mL,1750,1,9.41,4466,AMERICAN VINTAGE BEVERAGE
1,5255,TGI Fridays Ultimte Mudslide,12.99,1750mL,1750,1,9.35,4466,AMERICAN VINTAGE BEVERAGE
2,3140,TGI Fridays Orange Dream,14.99,1750mL,1750,1,11.19,4466,AMERICAN VINTAGE BEVERAGE


In [ ]:
vendor_invoice = pd.read_sql_query(f"select * from vendor_invoice where VendorNumber = 4466",conn)
vendor_invoice[:5]

,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57,None
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-19,8207,2023-12-27,2024-02-26,335,3142.33,16.97,None
2,4466,AMERICAN VINTAGE BEVERAGE,2024-01-18,8307,2024-01-03,2024-02-18,41,383.35,1.99,None
3,4466,AMERICAN VINTAGE BEVERAGE,2024-01-27,8469,2024-01-14,2024-03-11,72,673.20,3.30,None
4,4466,AMERICAN VINTAGE BEVERAGE,2024-02-04,8532,2024-01-19,2024-03-15,79,740.21,3.48,None


In [ ]:
sales = pd.read_sql_query(f"""select * from sales where VendorNo = 4466""",conn)
sales

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-09,1750.0,1,1.84,4466.0,AMERICAN VINTAGE BEVERAGE
1,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-12,1750.0,1,1.84,4466.0,AMERICAN VINTAGE BEVERAGE
2,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-15,1750.0,1,1.84,4466.0,AMERICAN VINTAGE BEVERAGE
3,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-21,1750.0,1,1.84,4466.0,AMERICAN VINTAGE BEVERAGE
4,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-23,1750.0,1,1.84,4466.0,AMERICAN VINTAGE BEVERAGE
5,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-31,1750.0,1,1.84,4466.0,AMERICAN VINTAGE BEVERAGE
6,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,1,12.99,12.99,2024-01-02,1750.0,1,1.84,4466.0,AMERICAN VINTAGE BEVERAGE
7,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,1,12.99,12.99,2024-01-07,1750.0,1,1.84,4466.0,AMERICAN VINTAGE BEVERAGE
8,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,1,12.99,12.99,2024-01-14,1750.0,1,1.84,4466.0,AMERICAN VINTAGE BEVERAGE
9,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,2,25.98,12.99,2024-01-15,1750.0,1,3.67,4466.0,AMERICAN VINTAGE BEVERAGE


In [ ]:
purchases

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,4,37.40,1
1,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,9.41,5,47.05,1
2,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,6,56.10,1
3,38_GOULCREST_5215,38,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-07,2024-01-19,2024-02-26,9.41,6,56.46,1
4,59_CLAETHORPES_5215,59,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-05,2024-01-19,2024-02-26,9.41,6,56.46,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,81_PEMBROKE_5215,81,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-29,2025-01-04,2025-02-10,9.41,6,56.46,1
2188,62_KILMARNOCK_5255,62,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-28,2025-01-04,2025-02-10,9.35,5,46.75,1
2189,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-28,2025-01-04,2025-02-10,9.41,5,47.05,1
2190,6_GOULCREST_5215,6,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-31,2025-01-04,2025-02-10,9.41,6,56.46,1


In [ ]:
purchases.groupby(['Brand','PurchasePrice'])[['Quantity','Dollars']].sum()

,,Quantity,Dollars
Brand,PurchasePrice,,
3140,11.19,4640,51921.60
5215,9.41,4923,46325.43
5255,9.35,6215,58110.25


In [ ]:
purchase_prices

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,5215,TGI Fridays Long Island Iced,12.99,1750mL,1750,1,9.41,4466,AMERICAN VINTAGE BEVERAGE
1,5255,TGI Fridays Ultimte Mudslide,12.99,1750mL,1750,1,9.35,4466,AMERICAN VINTAGE BEVERAGE
2,3140,TGI Fridays Orange Dream,14.99,1750mL,1750,1,11.19,4466,AMERICAN VINTAGE BEVERAGE


In [ ]:
vendor_invoice['PONumber'].nunique()

55

In [ ]:
sales[:5]

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-09,1750.0,1,1.84,4466.0,AMERICAN VINTAGE BEVERAGE
1,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-12,1750.0,1,1.84,4466.0,AMERICAN VINTAGE BEVERAGE
2,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-15,1750.0,1,1.84,4466.0,AMERICAN VINTAGE BEVERAGE
3,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-21,1750.0,1,1.84,4466.0,AMERICAN VINTAGE BEVERAGE
4,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-23,1750.0,1,1.84,4466.0,AMERICAN VINTAGE BEVERAGE


In [ ]:
sales.groupby('Brand')[['SalesDollars','SalesPrice','SalesQuantity']].sum()

,SalesDollars,SalesPrice,SalesQuantity
Brand,,,
5215,376.71,337.74,29
5255,558.57,428.67,43


In [ ]:
vendor_invoice.columns

Index(['VendorNumber', 'VendorName', 'InvoiceDate', 'PONumber', 'PODate',
       'PayDate', 'Quantity', 'Dollars', 'Freight', 'Approval'],
      dtype='object')

In [ ]:
frieght_summary =pd.read_sql_query(""" select VendorNumber, SUM(Freight) as freightcost from Vendor_invoice  GROUP BY VendorNumber """,conn)

In [ ]:
frieght_summary

,VendorNumber,freightcost
0,2,27.08
1,54,0.48
2,60,367.52
3,105,62.39
4,200,6.19
...,...,...
121,98450,856.02
122,99166,130.09
123,172662,178.34
124,173357,202.50


In [ ]:
purchases


,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,4,37.40,1
1,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,9.41,5,47.05,1
2,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,6,56.10,1
3,38_GOULCREST_5215,38,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-07,2024-01-19,2024-02-26,9.41,6,56.46,1
4,59_CLAETHORPES_5215,59,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-05,2024-01-19,2024-02-26,9.41,6,56.46,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,81_PEMBROKE_5215,81,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-29,2025-01-04,2025-02-10,9.41,6,56.46,1
2188,62_KILMARNOCK_5255,62,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-28,2025-01-04,2025-02-10,9.35,5,46.75,1
2189,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-28,2025-01-04,2025-02-10,9.41,5,47.05,1
2190,6_GOULCREST_5215,6,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-31,2025-01-04,2025-02-10,9.41,6,56.46,1


In [ ]:
pd.read_sql_query(""" select
p.VendorNumber,
p.VendorName,
p.Brand,
p.PurchasePrice,
pp.Volume,
pp.Price as actual_price,
SUM(p.Quantity) as totalpurchasequantity,
SUM(p.Dollars) as totalpurchasedollars
from purchases p
join purchase_prices pp
on p.Brand == PP.Brand
group by p.VendorNumber , p.VendorName, p.Brand
order by totalpurchasedollars
 """, conn)

,VendorNumber,VendorName,Brand,PurchasePrice,Volume,actual_price,totalpurchasequantity,totalpurchasedollars


In [ ]:
pd.read_sql_query("""
SELECT
  p.VendorNumber,
  p.VendorName,
  p.Brand,
  p.PurchasePrice,
  pp.Volume,
  pp.Price AS ActualPrice,
  SUM(p.Quantity) AS TotalPurchaseQuantity,
  SUM(p.Dollars) AS TotalPurchaseDollars
FROM purchases p
JOIN purchase_prices pp
  ON p.Brand = pp.Brand
WHERE p.PurchasePrice > 0
GROUP BY p.VendorNumber, p.VendorName, p.Brand
ORDER BY TotalPurchaseDollars
""", conn)

,VendorNumber,VendorName,Brand,PurchasePrice,Volume,ActualPrice,TotalPurchaseQuantity,TotalPurchaseDollars
0,7245,PROXIMO SPIRITS INC.,3065,0.71,50,0.99,1,0.71
1,3960,DIAGEO NORTH AMERICA INC,6127,1.47,200,1.99,1,1.47
2,3924,HEAVEN HILL DISTILLERIES,9123,0.74,50,0.99,2,1.48
3,8004,SAZERAC CO INC,5683,0.39,50,0.49,6,2.34
4,9815,WINE GROUP INC,8527,1.32,750,4.99,2,2.64
...,...,...,...,...,...,...,...,...
10687,3960,DIAGEO NORTH AMERICA INC,3545,21.89,1750,29.99,138109,3023206.01
10688,3960,DIAGEO NORTH AMERICA INC,4261,16.17,1750,22.99,201682,3261197.94
10689,17035,PERNOD RICARD USA,8068,18.24,1750,24.99,187407,3418303.68
10690,4425,MARTIGNETTI COMPANIES,3405,23.19,1750,28.99,164038,3804041.22


In [ ]:
sales.columns

Index(['InventoryId', 'Store', 'Brand', 'Description', 'Size', 'SalesQuantity',
       'SalesDollars', 'SalesPrice', 'SalesDate', 'Volume', 'Classification',
       'ExciseTax', 'VendorNo', 'VendorName'],
      dtype='object')

In [ ]:
sales[:5]

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-09,1750.0,1,1.84,4466.0,AMERICAN VINTAGE BEVERAGE
1,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-12,1750.0,1,1.84,4466.0,AMERICAN VINTAGE BEVERAGE
2,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-15,1750.0,1,1.84,4466.0,AMERICAN VINTAGE BEVERAGE
3,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-21,1750.0,1,1.84,4466.0,AMERICAN VINTAGE BEVERAGE
4,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-23,1750.0,1,1.84,4466.0,AMERICAN VINTAGE BEVERAGE


In [ ]:
pd.read_sql_query("""
select
VendorNo,
Brand,
SUM(SalesDollars) as totalsalesdollar,
SUM(SalesPrice) as totalsalesprice,
SUM(ExciseTax) as totalexcisetax

from sales
group by VendorNo, Brand
order by totalsalesdollar
""",conn)

,VendorNo,Brand,totalsalesdollar,totalsalesprice,totalexcisetax
0,8004.0,5287,0.49,0.49,0.05
1,3960.0,8065,0.99,0.99,0.05
2,7245.0,3053,0.99,0.99,0.05
3,3960.0,4173,1.98,0.99,0.10
4,1392.0,18000,3.99,3.99,0.22
...,...,...,...,...,...
4581,4425.0,3405,31279.57,8367.21,1916.28
4582,17035.0,8068,39533.70,8256.81,2811.08
4583,3960.0,3545,47408.27,8359.23,2890.18
4584,1128.0,1233,51825.60,11480.81,2645.68


In [ ]:
import time
start = time.time()
final_table  = pd.read_sql_query("""
select
pp.VendorNumber,
pp.Brand,
pp.Price,
pp. PurchasePrice,
SUM(s.SalesQuantity) as totalsalesquantity,
SUM(s.SalesDollars) as totalsalesdollar,
SUM(s.SalesPrice) as totalsalesprice,
SUM(s.ExciseTax) as totalexcisetax,
SUM(vi.Quantity) AS TotalPurchaseQuantity,
SUM(vi.Dollars) AS TotalPurchaseDollars,
SUM(vi.Freight) as totalfreightcost

from purchase_prices pp
join sales s
  on pp.VendorNumber = s.VendorNo
  AND pp.brand = s.Brand
join vendor_invoice vi
  on pp.Vendornumber = vi.VendorNumber
group by pp.VendorNumber,pp.Brand,pp.Price,pp.PurchasePrice

""",conn)
end = time.time()

In [ ]:
final_table

,VendorNumber,Brand,Price,PurchasePrice,totalsalesquantity,totalsalesdollar,totalsalesprice,totalexcisetax,TotalPurchaseQuantity,TotalPurchaseDollars,totalfreightcost
0,105,8412,49.99,35.71,80,3999.20,3999.20,63.20,664,2.341240e+04,124.78
1,105,8419,6.99,5.30,200,1398.00,1398.00,20.00,1660,5.853100e+04,311.95
2,388,2034,32.99,21.32,1166,33802.34,32265.87,920.61,38787,8.634427e+05,4446.54
3,388,2119,89.99,66.40,53,4504.47,4504.47,41.87,1847,4.111632e+04,211.74
4,480,126,32.99,20.14,20680,659463.20,207836.20,21714.00,165540700,2.044428e+09,10357207.32
...,...,...,...,...,...,...,...,...,...,...,...
4580,98450,4145,19.99,12.68,4806,81653.94,43120.62,8832.78,491761,7.942700e+06,40232.94
4581,98450,7890,29.99,22.21,810,24291.90,16194.60,638.28,104630,1.689936e+06,8560.20
4582,98450,8543,19.99,14.81,432,8635.68,4317.84,340.20,41852,6.759744e+05,3424.08
4583,172662,4215,25.99,19.40,174,4522.26,753.71,136.88,1629,3.470803e+04,178.34


In [ ]:
vendorsalessummary = pd.read_sql_query("""
with freightsummary as (

  select
  VendorNumber,
  SUM(Freight) as frightcost
  from vendor_invoice
  group by VendorNumber
),

PurchaseSummary as (
select
  p.VendorNumber,
  p.VendorName,
  p.Brand,
  p.PurchasePrice,
  pp.Volume,
  pp.Price AS ActualPrice,
  SUM(p.Quantity) AS TotalPurchaseQuantity,
  SUM(p.Dollars) AS TotalPurchaseDollars
FROM purchases p
JOIN purchase_prices pp
  ON p.Brand = pp.Brand
WHERE p.PurchasePrice > 0
GROUP BY p.VendorNumber, p.VendorName, p.Brand, p.Description, p.PurchasePrice, pp.Price, pp.Volume
),

SalesSummary as (
  select
  VendorNo,
  Brand,
  SUM(SalesQuantity) as totalsalesquantity,
SUM(SalesDollars) as totalsalesdollar,
SUM(SalesPrice) as totalsalesprice,
SUM(ExciseTax) as totalexcisetax
from sales
group by VendorNo,Brand
)

select
  ps.VendorNumber,
  ps.VendorName,
  ps.Brand,
  ps.Description,
  ps.PurchasePrice,
  ps.Volume,
  ps.TotalPurchaseQuantity,
  ps.TotalPurchaseDollars,
  ss.TotalSalesDollars,
  ss.TotalSalesPrice,
  ss.TotalExciseTax,
  fs.FreightCost

from PurchaseSummary ps
left join SalesSummary ss
on ps.VendorNumber = ss. VendorNo
and ps.Brand = ss.Brand
left join FreightSummary fs
on ps.VendorNumber = fs.VendorNumber
order by ps.TotalPurchaseDollars desc


""",conn)

DatabaseError: Execution failed on sql '
with freightsummary as (

  select
  VendorNumber,
  SUM(Freight) as frightcost
  from vendor_invoice
  group by VendorNumber
),

PurchaseSummary as (
select
  p.VendorNumber,
  p.VendorName,
  p.Brand,
  p.PurchasePrice,
  pp.Volume,
  pp.Price AS ActualPrice,
  SUM(p.Quantity) AS TotalPurchaseQuantity,
  SUM(p.Dollars) AS TotalPurchaseDollars
FROM purchases p
JOIN purchase_prices pp
  ON p.Brand = pp.Brand
WHERE p.PurchasePrice > 0
GROUP BY p.VendorNumber, p.VendorName, p.Brand, p.Description, p.PurchasePrice, pp.Price, pp.Volume
),

SalesSummary as (
  select
  VendorNo,
  Brand,
  SUM(SalesQuantity) as totalsalesquantity,
SUM(SalesDollars) as totalsalesdollar,
SUM(SalesPrice) as totalsalesprice,
SUM(ExciseTax) as totalexcisetax
from sales 
group by VendorNo,Brand
)

select 
  ps.VendorNumber,
  ps.VendorName,
  ps.Brand,
  ps.Description,
  ps.PurchasePrice,
  ps.Volume,
  ps.TotalPurchaseQuantity,
  ps.TotalPurchaseDollars,
  ss.TotalSalesD

In [ ]:
import pandas as pd

vendor_sales_summary = pd.read_sql_query("""
  WITH FreightSummary AS (
    SELECT VendorNumber, SUM(Freight) AS FreightCost
    FROM vendor_invoice
    GROUP BY VendorNumber
  ),
  PurchaseSummary AS (
    SELECT
      p.VendorNumber,
      p.VendorName,
      p.Brand,
      p.PurchasePrice,
      pp.Volume,
      pp.Price AS ActualPrice,
      SUM(p.Quantity) AS TotalPurchaseQuantity,
      SUM(p.Dollars) AS TotalPurchaseDollars
    FROM purchases p
    JOIN purchase_prices pp ON p.Brand = pp.Brand
    WHERE p.PurchasePrice > 0
    GROUP BY p.VendorNumber, p.VendorName, p.Brand, p.PurchasePrice, pp.Volume, pp.Price
  ),
  SalesSummary AS (
    SELECT
      VendorNo,
      Brand,
      SUM(SalesDollars) AS TotalSalesDollars,
      AVG(SalesPrice) AS AvgSalesPrice,
      SUM(SalesQuantity) AS TotalSalesQuantity,
      SUM(ExciseTax) AS TotalExciseTax
    FROM sales
    GROUP BY VendorNo, Brand
  )

  SELECT
    ps.VendorNumber,
    ps.VendorName,
    ps.Brand,
    ps.PurchasePrice,
    ps.ActualPrice,
    ps.Volume,
    ps.TotalPurchaseQuantity,
    ps.TotalPurchaseDollars,
    ss.TotalSalesQuantity,
    ss.TotalSalesDollars,
    ss.AvgSalesPrice,
    ss.TotalExciseTax,
    fs.FreightCost
  FROM PurchaseSummary ps
  LEFT JOIN SalesSummary ss
    ON ps.VendorNumber = ss.VendorNo AND ps.Brand = ss.Brand
  LEFT JOIN FreightSummary fs
    ON ps.VendorNumber = fs.VendorNumber
  ORDER BY ps.TotalPurchaseDollars DESC
""", conn)

In [18]:


vendor_sales_summary = pd.read_sql_query("""
  WITH FreightSummary AS (
    SELECT VendorNumber, SUM(Freight) AS FreightCost
    FROM vendor_invoice
    GROUP BY VendorNumber
  ),
  PurchaseSummary AS (
    SELECT
      p.VendorNumber,
      p.VendorName,
      p.Brand,
      p.Description,
      p.PurchasePrice,
      pp.Volume,
      pp.Price AS ActualPrice,
      SUM(p.Quantity) AS TotalPurchaseQuantity,
      SUM(p.Dollars) AS TotalPurchaseDollars
    FROM purchases p
    JOIN purchase_prices pp ON p.Brand = pp.Brand
    WHERE p.PurchasePrice > 0
    GROUP BY p.VendorNumber, p.VendorName, p.Brand, p.PurchasePrice, pp.Volume, pp.Price
  ),
  SalesSummary AS (
    SELECT
      VendorNo,
      Brand,
      SUM(SalesDollars) AS TotalSalesDollars,
      AVG(SalesPrice) AS AvgSalesPrice,
      SUM(SalesQuantity) AS TotalSalesQuantity,
      SUM(ExciseTax) AS TotalExciseTax
    FROM sales
    GROUP BY VendorNo, Brand
  )

  SELECT
    ps.VendorNumber,
    ps.VendorName,
    ps.Brand,
    ps.Description,
    ps.PurchasePrice,
    ps.ActualPrice,
    ps.Volume,
    ps.TotalPurchaseQuantity,
    ps.TotalPurchaseDollars,
    ss.TotalSalesQuantity,
    ss.TotalSalesDollars,
    ss.AvgSalesPrice,
    ss.TotalExciseTax,
    fs.FreightCost
  FROM PurchaseSummary ps
  LEFT JOIN SalesSummary ss
    ON ps.VendorNumber = ss.VendorNo AND ps.Brand = ss.Brand
  LEFT JOIN FreightSummary fs
    ON ps.VendorNumber = fs.VendorNumber
  ORDER BY ps.TotalPurchaseDollars DESC
""", conn)

In [ ]:
vendor_sales_summary

,VendorNumber,VendorName,Brand,Description,PurchasePrice,ActualPrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,AvgSalesPrice,TotalExciseTax,FreightCost
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,26.27,36.99,1750,145080,3811251.60,1440.0,51825.60,35.990000,2645.68,68601.68
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,23.19,28.99,1750,164038,3804041.22,1043.0,31279.57,29.990000,1916.28,144929.24
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,18.24,24.99,1750,187407,3418303.68,1530.0,39533.70,25.883417,2811.08,123780.22
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,16.17,22.99,1750,201682,3261197.94,2974.0,65398.26,21.990000,5464.51,257032.07
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,21.89,29.99,1750,138109,3023206.01,1573.0,47408.27,30.177726,2890.18,257032.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,Concannon Glen Ellen Wh Zin,1.32,4.99,750,2,2.64,NaN,NaN,NaN,NaN,27100.41
10688,8004,SAZERAC CO INC,5683,Dr McGillicuddy's Apple Pie,0.39,0.49,50,6,2.34,NaN,NaN,NaN,NaN,50293.62
10689,3924,HEAVEN HILL DISTILLERIES,9123,Deep Eddy Vodka,0.74,0.99,50,2,1.48,NaN,NaN,NaN,NaN,14069.87
10690,3960,DIAGEO NORTH AMERICA INC,6127,The Club Strawbry Margarita,1.47,1.99,200,1,1.47,NaN,NaN,NaN,NaN,257032.07


**This query generates a vendor-wise sales and purchase summary which is valueable for:**
* the query involves heavy joins and aggregations on large datasets like sales and purchases
* helps in analyzing sales, purchasings and pricing for differnt  vendor and brands
* instead of running expensive queries each time, dashboards can fetch data quickly from vendor_sales_summary


In [20]:
vendor_sales_summary.dtypes

,0
VendorNumber,int64
VendorName,object
Brand,int64
Description,object
PurchasePrice,float64
ActualPrice,float64
Volume,float64
TotalPurchaseQuantity,int64
TotalPurchaseDollars,float64
TotalSalesQuantity,float64


In [22]:
vendor_sales_summary.isnull().sum()

,0
VendorNumber,0
VendorName,0
Brand,0
Description,0
PurchasePrice,0
ActualPrice,0
Volume,0
TotalPurchaseQuantity,0
TotalPurchaseDollars,0
TotalSalesQuantity,0


In [25]:

vendor_sales_summary['VendorName'].unique()

array(['BROWN-FORMAN CORP', 'MARTIGNETTI COMPANIES', 'PERNOD RICARD USA',
       'DIAGEO NORTH AMERICA INC', 'BACARDI USA INC',
       'JIM BEAM BRANDS COMPANY', 'MAJESTIC FINE WINES',
       'ULTRA BEVERAGE COMPANY LLP', 'STOLI GROUP,(USA) LLC',
       'PROXIMO SPIRITS INC.', 'MOET HENNESSY USA INC', 'CAMPARI AMERICA',
       'SAZERAC CO INC', 'CONSTELLATION BRANDS INC', 'M S WALKER INC',
       'SAZERAC NORTH AMERICA INC.', 'PALM BAY INTERNATIONAL INC',
       'REMY COINTREAU USA INC', 'SIDNEY FRANK IMPORTING CO',
       'E & J GALLO WINERY', 'WILLIAM GRANT & SONS INC',
       'HEAVEN HILL DISTILLERIES', 'DISARONNO INTERNATIONAL LLC',
       'EDRINGTON AMERICAS', 'CASTLE BRANDS CORP.',
       'SOUTHERN WINE & SPIRITS NE', 'STE MICHELLE WINE ESTATES',
       'TRINCHERO FAMILY ESTATES', 'MHW LTD', 'WINE GROUP INC',
       'PERFECTA WINES', 'LUXCO INC', 'TREASURY WINE ESTATES',
       'DIAGEO CHATEAU ESTATE WINES', 'SHAW ROSS INT L IMP LTD',
       'PINE STATE TRADING CO', 'PHILLIPS PRO

In [19]:
vendor_sales_summary['Volume'] = vendor_sales_summary['Volume'].astype('float64')

In [21]:
vendor_sales_summary.fillna(0,inplace=True)

In [24]:
vendor_sales_summary['VendorName'] = vendor_sales_summary['VendorName'].str.strip()

In [26]:
vendor_sales_summary['grossprofit'] = vendor_sales_summary['TotalSalesDollars'] - vendor_sales_summary['TotalPurchaseDollars']

In [28]:
vendor_sales_summary['grossprofit'].min()

-3772761.649999954

In [29]:
vendor_sales_summary['profitmargin'] =( vendor_sales_summary['grossprofit'] /  vendor_sales_summary['TotalSalesDollars']) * 100

In [32]:
vendor_sales_summary['stockturnover'] = vendor_sales_summary['TotalSalesQuantity'] / vendor_sales_summary['TotalPurchaseQuantity']

In [33]:
vendor_sales_summary['salestopurchaseratio'] = vendor_sales_summary['TotalSalesDollars'] / vendor_sales_summary['TotalPurchaseDollars']

In [34]:
vendor_sales_summary

,VendorNumber,VendorName,Brand,Description,PurchasePrice,ActualPrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,AvgSalesPrice,TotalExciseTax,FreightCost,grossprofit,profitmargin,stockturnover,salestopurchaseratio
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,26.27,36.99,1750.0,145080,3811251.60,1440.0,51825.60,35.990000,2645.68,68601.68,-3759426.00,-7.253994e+03,0.009926,0.013598
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,23.19,28.99,1750.0,164038,3804041.22,1043.0,31279.57,29.990000,1916.28,144929.24,-3772761.65,-1.206142e+04,0.006358,0.008223
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,18.24,24.99,1750.0,187407,3418303.68,1530.0,39533.70,25.883417,2811.08,123780.22,-3378769.98,-8.546556e+03,0.008164,0.011565
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,16.17,22.99,1750.0,201682,3261197.94,2974.0,65398.26,21.990000,5464.51,257032.07,-3195799.68,-4.886674e+03,0.014746,0.020053
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,21.89,29.99,1750.0,138109,3023206.01,1573.0,47408.27,30.177726,2890.18,257032.07,-2975797.74,-6.276959e+03,0.011390,0.015681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,Concannon Glen Ellen Wh Zin,1.32,4.99,750.0,2,2.64,0.0,0.00,0.000000,0.00,27100.41,-2.64,-inf,0.000000,0.000000
10688,8004,SAZERAC CO INC,5683,Dr McGillicuddy's Apple Pie,0.39,0.49,50.0,6,2.34,0.0,0.00,0.000000,0.00,50293.62,-2.34,-inf,0.000000,0.000000
10689,3924,HEAVEN HILL DISTILLERIES,9123,Deep Eddy Vodka,0.74,0.99,50.0,2,1.48,0.0,0.00,0.000000,0.00,14069.87,-1.48,-inf,0.000000,0.000000
10690,3960,DIAGEO NORTH AMERICA INC,6127,The Club Strawbry Margarita,1.47,1.99,200.0,1,1.47,0.0,0.00,0.000000,0.00,257032.07,-1.47,-inf,0.000000,0.000000


In [35]:
cursor = conn.cursor()


In [40]:
cursor.execute ("""CREATE TABLE vendor_sales_summary (
    VendorNumber INT,
    VendorName varchar(100),
    Brand INT,
    Description varchar(100),
    PurchasePrice decimal(10,2),
    ActualPrice decimal(10,2),
    Volume int ,
    TotalPurchaseQuantity INT,
    TotalPurchaseDollars decimal(15,2),
    TotalSalesQuantity INT,
    TotalSalesDollars decimal(15,2),
    AvgSalesPrice decimal(15,2),
    TotalExciseTax decimal(15,2),
    FreightCost decimal(15,2),
    grossprofit decimal(15,2),
    profitmargin decimal(15,2),
    stockturnover decimal(15,2),
    salestopurchaseratio decimal(15,2),

    primary key (vendorNumber,Brand)
);
"""
)

In [44]:
pd.read_sql_query(""" select * from vendor_sales_summary  """,conn)

,VendorNumber,VendorName,Brand,Description,PurchasePrice,ActualPrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,AvgSalesPrice,TotalExciseTax,FreightCost,grossprofit,profitmargin,stockturnover,salestopurchaseratio
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,26.27,36.99,1750.0,145080,3811251.60,1440.0,51825.60,35.990000,2645.68,68601.68,-3759426.00,-7.253994e+03,0.009926,0.013598
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,23.19,28.99,1750.0,164038,3804041.22,1043.0,31279.57,29.990000,1916.28,144929.24,-3772761.65,-1.206142e+04,0.006358,0.008223
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,18.24,24.99,1750.0,187407,3418303.68,1530.0,39533.70,25.883417,2811.08,123780.22,-3378769.98,-8.546556e+03,0.008164,0.011565
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,16.17,22.99,1750.0,201682,3261197.94,2974.0,65398.26,21.990000,5464.51,257032.07,-3195799.68,-4.886674e+03,0.014746,0.020053
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,21.89,29.99,1750.0,138109,3023206.01,1573.0,47408.27,30.177726,2890.18,257032.07,-2975797.74,-6.276959e+03,0.011390,0.015681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,Concannon Glen Ellen Wh Zin,1.32,4.99,750.0,2,2.64,0.0,0.00,0.000000,0.00,27100.41,-2.64,-inf,0.000000,0.000000
10688,8004,SAZERAC CO INC,5683,Dr McGillicuddy's Apple Pie,0.39,0.49,50.0,6,2.34,0.0,0.00,0.000000,0.00,50293.62,-2.34,-inf,0.000000,0.000000
10689,3924,HEAVEN HILL DISTILLERIES,9123,Deep Eddy Vodka,0.74,0.99,50.0,2,1.48,0.0,0.00,0.000000,0.00,14069.87,-1.48,-inf,0.000000,0.000000
10690,3960,DIAGEO NORTH AMERICA INC,6127,The Club Strawbry Margarita,1.47,1.99,200.0,1,1.47,0.0,0.00,0.000000,0.00,257032.07,-1.47,-inf,0.000000,0.000000


In [43]:
vendor_sales_summary.to_sql('vendor_sales_summary',conn,if_exists='replace', index=False)

10692